In [8]:
# exploration_orders.ipynb

import duckdb
import pandas as pd
import matplotlib.pyplot as plt

# Connect to existing DuckDB database
con = duckdb.connect("instacart.duckdb")

# Preview order structure
df_orders = con.execute("SELECT * FROM orders LIMIT 10").fetchdf()
df_orders
tables = con.execute("SHOW TABLES").fetchdf()["name"].tolist()
for table in tables:
    print(f"--- {table} ---")
    print(con.execute(f"DESCRIBE {table}").fetchdf())



--- aisles ---
  column_name column_type null   key default extra
0    aisle_id      BIGINT  YES  None    None  None
1       aisle     VARCHAR  YES  None    None  None
--- departments ---
     column_name column_type null   key default extra
0  department_id      BIGINT  YES  None    None  None
1     department     VARCHAR  YES  None    None  None
--- order_products_prior ---
         column_name column_type null   key default extra
0           order_id      BIGINT  YES  None    None  None
1         product_id      BIGINT  YES  None    None  None
2  add_to_cart_order      BIGINT  YES  None    None  None
3          reordered      BIGINT  YES  None    None  None
--- order_products_train ---
         column_name column_type null   key default extra
0           order_id      BIGINT  YES  None    None  None
1         product_id      BIGINT  YES  None    None  None
2  add_to_cart_order      BIGINT  YES  None    None  None
3          reordered      BIGINT  YES  None    None  None
--- orders -

In [10]:

# 1. Top Selling Products
df_user_orders2 = con.execute("""
   SELECT 
        o.order_id,
        o.user_id,
        o.order_dow,
        o.order_hour_of_day,
        p.product_name,
        opp.add_to_cart_order,
        opp.reordered
    FROM orders o
    JOIN order_products_prior opp
        ON o.order_id = opp.order_id
    JOIN products p
        ON opp.product_id = p.product_id
    WHERE o.user_id = 155
;""").fetchdf()
print(df_user_orders2)




     order_id  user_id  order_dow order_hour_of_day            product_name  \
0     3103333      155          1                14   Medium Salsa Organica   
1     2772122      155          0                08   Medium Salsa Organica   
2     2402663      155          0                11   Medium Salsa Organica   
3     1253654      155          0                21  100% Raw Coconut Water   
4      938110      155          2                18  100% Raw Coconut Water   
..        ...      ...        ...               ...                     ...   
236   2839132      155          5                02    Organic Hass Avocado   
237   1997753      155          2                22    Organic Hass Avocado   
238   1296427      155          4                15    Organic Hass Avocado   
239    122604      155          4                20    Organic Hass Avocado   
240    119858      155          2                11    Organic Hass Avocado   

     add_to_cart_order  reordered  
0              

In [ ]:

# 2. Most Popular Aisles
#df_user_orders = con.execute("""
#    SELECT *
#    FROM orders
#    LIMIT 10
#""").fetchdf()
#print(df_user_orders)

# 3. Which Day of the Week Do People Order Most?
#df_user_orders = con.execute("""
#    SELECT *
#    FROM orders
#    LIMIT 10
#""").fetchdf()
#print(df_user_orders)

# 4. What Time of Day Do People Order?
# df_user_orders = con.execute("""
#     SELECT *
#     FROM orders
#     LIMIT 10
# """).fetchdf()
# print(df_user_orders)

# 5. Top Departments by Volume
# df_user_orders = con.execute("""
#     SELECT *
#     FROM orders
#     LIMIT 10
# """).fetchdf()
# print(df_user_orders)

